In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install swifter
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from ipywidgets import interact, fixed
from scipy import spatial

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.patches as patches
from matplotlib import animation
from matplotlib.animation import FFMpegWriter
pd.set_option('max_columns', 100)
import swifter
import dateutil
from math import radians
from IPython.display import Video
import types 

import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
weeks = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if 'week' in filename: 
            weeks.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
weeks.sort()
df = pd.read_csv(weeks[0])
df.head()

In [ ]:
week = df.query('gameId == 2018090600')
week.head()

In [ ]:
weekArray = np.array(week)
weekMod = pd.DataFrame(weekArray, columns=week.columns)
groupedWeek =weekMod.groupby(['gameId', 'playId', 'frameId'])
playerXY = {}
for name, group in groupedWeek:
    playerXY[name] = []
    for row in group.iterrows():
        data = [row[1]['nflId'], row[1]['team'], row[1]['x'], row[1]['y'], row[1]['dir']]
        playerXY[name].append(data)

In [ ]:
features = list(weekMod.columns)
weekArray = np.array(weekMod)
minOppDist = []

## Additional variables

From basic field theory we need aditional variables 
* distance to ball
* speed of specific players


# Computing distances

In [ ]:
for player in weekArray:        
    if player[features.index('team')] != 'football':
        opponentPositions = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
        distances = []
        directions = []
        opponents = []
        xs = []
        ys = []
        ball_dist = None
        for oppPos in opponentPositions: 
            if player[features.index('team')] != oppPos[1] and player[features.index('team')] != 'football' and oppPos[1] != 'football':
                dx = (player[features.index('x')] - oppPos[2])**2
                dy = (player[features.index('y')] - oppPos[3])**2
                dist = np.sqrt(dx+dy)
                distances.append(dist)
                directions.append(oppPos[4])
                opponents.append(oppPos[0])
                xs.append(oppPos[2])
                ys.append(oppPos[3])
            if oppPos[1] == 'football':
                dx = (player[features.index('x')] - oppPos[2])**2
                dy = (player[features.index('y')] - oppPos[3])**2
                ball_dist = np.sqrt(dx+dy)
        minDist = min(distances)
        closestOpponent = opponents[np.argmin(distances)]
        opponentDir = directions[np.argmin(distances)]
        opponentX = xs[np.argmin(distances)]
        opponentY = ys[np.argmin(distances)]
        summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist, closestOpponent, opponentDir, opponentX, opponentY, ball_dist]
        minOppDist.append(summary)
             
minOppDist = pd.DataFrame(minOppDist, columns=['gameId', 'playId', 'frameId', 'nflId', 'oppMinDist', 'closestOpp(nflId)', 'oppDir', 'oppX', 'oppY', 'ball_dist'])
weekMod = pd.merge(weekMod, minOppDist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])

## Compute Speed


In [ ]:
def compute_speed(data):
    data['v_x'] = np.sin(data['dir']*np.pi/180) * data['s']
    data['v_y'] = np.cos(data['dir']*np.pi/180) * data['s']
    data['v_theta'] = np.arctan(data['v_x']/data['v_y']) if data['v_y'] != 0 else 0 
    return data
def speed_ratio(data):
    data['s_ratio'] = data['s']/13
    return data
def next_loc(data):
    delta_t= 0.2
    data['x_next'] = data.x + data['v_x']*delta_t
    data['y_next'] = data.y + data['v_y']*delta_t
    return data

def radiusOfInfluence(data):
    min_radius = 2.00
    max_radius = 10.00
    max_distance_from_ball = 20.00
    data['radius_of_influence'] = min_radius + data['ball_dist']**3 * (max_radius-min_radius) / max_distance_from_ball if min_radius + data['ball_dist']**3 * (max_radius-min_radius) / max_distance_from_ball<max_radius else max_radius
    return data

In [ ]:
weekMod = weekMod.swifter.set_npartitions(10).apply(compute_speed, axis=1)
weekMod = weekMod.swifter.set_npartitions(10).apply(speed_ratio, axis=1)
weekMod = weekMod.swifter.set_npartitions(10).apply(next_loc, axis=1)
weekMod = weekMod.swifter.set_npartitions(10).apply(radiusOfInfluence,axis=1)

In [ ]:
def rotMat(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array(((c, -s), (s, c)))
def scalingMat(rad_of_influence, s_ratio):
    return np.array([[rad_of_influence*(1+s_ratio),0], [0, rad_of_influence*(1-s_ratio)]])
def covMat(theta, rad_of_influence, s_ratio):
    R = rotMat(theta)
    S = scalingMat(rad_of_influence, s_ratio)
    return R @S @S @ np.linalg.inv(R)

def multi_var_gaussian(pos, mu, sigma):
    n = mu.shape[0]
    sigma_det = np.linalg.det(sigma)
    sigma_inv = np.linalg.inv(sigma)
    N = np.sqrt((2*np.pi)**n *sigma_det)
    fac = np.einsum('...k,kl,...l->...', pos-mu, sigma_inv, pos-mu)
    return np.exp(-fac / 2) / N

def player_influence(theta, rad_of_influence, s_ratio, data):
    """
    grid should have a size of 120 and with of 53
    all other parameters come from influence stuff
    
    """
    x = np.linspace(0,120, 120)
    y = np.linspace(0,53,53)
    X,Y = np.meshgrid(x,y)
    
    pos = np.empty(X.shape + (2,))
    pos[:, :, 0] = X
    pos[:, :, 1] = Y
    sigma = covMat(theta, rad_of_influence, s_ratio)
    mu = np.array((data['x'], data['y']))
    Z = multi_var_gaussian(pos, mu,sigma)
    return Z 
def field_control(home, away):
    home_sum = np.zeros((53,120))
    away_sum = np.zeros((53,120))
    for i in home: 
        home_sum += i/np.sum(i)
    for i in away: 
        away_sum += i/np.sum(i)
    mapping = home_sum - away_sum
    return mapping, home_sum, away_sum

def timeframe(frame):
    home = []
    away = []
    for key, row in frame.iterrows():
        if row['team'] == 'home':
            home.append(player_influence(row['v_theta'], row['radius_of_influence'], row['s_ratio'], row))
        elif row['team'] == 'away':
            away.append(player_influence(row['v_theta'], row['radius_of_influence'], row['s_ratio'], row))
        else:
            pass
    mapping, home_sum, away_sum = field_control(home, away)
    return mapping, home_sum, away_sum


In [ ]:
play = weekMod.query('gameId== 2018090600 and playId==75')
print(max(play['frameId']))
play.head()

## Here you can play

Visualization not completely done select play above and timefroame below.

Paper: [Wide Open Space](https://www.researchgate.net/publication/324942294_Wide_Open_Spaces_A_statistical_technique_for_measuring_space_creation_in_professional_soccer)

In [ ]:
#### adapted from ar2017 -> Identifying coverage schemes among defensive backs

def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=55,
                          highlight_first_down_line=False,
                          yards_to_go=10,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        #plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
        #         color='yellow')
        
    if highlight_first_down_line:
        fl = hl + yards_to_go
        plt.plot([fl, fl], [0, 53.3], color='yellow')
        #plt.text(fl + 2, 50, '<- {}'.format(highlighted_name),
        #         color='yellow')
    return fig, ax

def calculate_dx_dy_arrow(x, y, angle, speed, multiplier):
    if angle <= 90:
        angle = angle
        dx = np.sin(radians(angle)) * multiplier * speed
        dy = np.cos(radians(angle)) * multiplier * speed
        return dx, dy
    if angle > 90 and angle <= 180:
        angle = angle - 90
        dx = np.sin(radians(angle)) * multiplier * speed
        dy = -np.cos(radians(angle)) * multiplier * speed
        return dx, dy
    if angle > 180 and angle <= 270:
        angle = angle - 180
        dx = -(np.sin(radians(angle)) * multiplier * speed)
        dy = -(np.cos(radians(angle)) * multiplier * speed)
        return dx, dy
    if angle > 270 and angle <= 360:
        angle = 360 - angle
        dx = -np.sin(radians(angle)) * multiplier * speed
        dy = np.cos(radians(angle)) * multiplier * speed
        return dx, dy

def animate_player_movement(weekData, playId, gameId):
    playData = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv')
    play = weekMod.query('gameId== '+str(gameId)+' and playId=='+ str(playId))
    playHome = weekData.query('gameId==' + str(gameId) + ' and playId==' + str(playId) + ' and team == "home"')
    playAway = weekData.query('gameId==' + str(gameId) + ' and playId==' + str(playId) + ' and team == "away"')
    playFootball = weekData.query('gameId==' + str(gameId) + ' and playId==' + str(playId) + ' and team == "football"')
    
    playHome['time'] = playHome['time'].apply(lambda x: dateutil.parser.parse(x).timestamp()).rank(method='dense')
    playAway['time'] = playAway['time'].apply(lambda x: dateutil.parser.parse(x).timestamp()).rank(method='dense')
    playFootball['time'] = playFootball['time'].apply(lambda x: dateutil.parser.parse(x).timestamp()).rank(method='dense')
    
    maxTime = int(playAway['time'].unique().max())
    minTime = int(playAway['time'].unique().min())
    
    yardlineNumber = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['yardlineNumber'].item()
    yardsToGo = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['yardsToGo'].item()
    absoluteYardlineNumber = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['absoluteYardlineNumber'].item() - 10
    playDir = playHome.sample(1)['playDirection'].item()
    
    if (absoluteYardlineNumber > 50):
        yardlineNumber = 100 - yardlineNumber
    if (absoluteYardlineNumber <= 50):
        yardlineNumber = yardlineNumber
        
    if (playDir == 'left'):
        yardsToGo = -yardsToGo
    else:
        yardsToGo = yardsToGo
    
    fig, ax = create_football_field(highlight_line=True, highlight_line_number=yardlineNumber, highlight_first_down_line=True, yards_to_go=yardsToGo)
    playDesc = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['playDescription'].item()
    plt.title(f'Game # {gameId} Play # {playId} \n {playDesc}')
    
    def update_animation(time):
        patch = []
        
        homeX = playHome.query('time == ' + str(time))['x']
        homeY = playHome.query('time == ' + str(time))['y']
        homeNum = playHome.query('time == ' + str(time))['jerseyNumber']
        homeOrient = playHome.query('time == ' + str(time))['o']
        homeDir = playHome.query('time == ' + str(time))['dir']
        homeSpeed = playHome.query('time == ' + str(time))['s']
        homePosition = playHome.query('time == ' + str(time))['position']
        patch.extend(plt.plot(homeX, homeY, 'o',c='gold', ms=20, mec='white', zorder=3))
        
        
        # Home players' jersey number 
        for x, y, num in zip(homeX, homeY, homeNum):
            patch.append(plt.text(x, y, int(num), va='center', ha='center', color='black', size='medium'))
            
        # Home players' orientation
        for x, y, orient in zip(homeX, homeY, homeOrient):
            dx, dy = calculate_dx_dy_arrow(x, y, orient, 1, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='gold', width=0.5, shape='full'))
            
        # Home players' direction
        for x, y, direction, speed in zip(homeX, homeY, homeDir, homeSpeed):
            dx, dy = calculate_dx_dy_arrow(x, y, direction, speed, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='black', width=0.25, shape='full'))
        
        
        awayX = playAway.query('time == ' + str(time))['x']
        awayY = playAway.query('time == ' + str(time))['y']
        awayNum = playAway.query('time == ' + str(time))['jerseyNumber']
        awayOrient = playAway.query('time == ' + str(time))['o']
        awayDir = playAway.query('time == ' + str(time))['dir']
        awaySpeed = playAway.query('time == ' + str(time))['s']
        awayPosition = playAway.query('time == ' + str(time))['position']
        patch.extend(plt.plot(awayX, awayY, 'o',c='orangered', ms=20, mec='white', zorder=3))
        
        # Away players' jersey number 
        for x, y, num in zip(awayX, awayY, awayNum):
            patch.append(plt.text(x, y, int(num), va='center', ha='center', color='white', size='medium'))
            
        # Away players' orientation
        for x, y, orient in zip(awayX, awayY, awayOrient):
            dx, dy = calculate_dx_dy_arrow(x, y, orient, 1, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='orangered', width=0.5, shape='full'))
        
        # Away players' direction
        for x, y, direction, speed in zip(awayX, awayY, awayDir, awaySpeed):
            dx, dy = calculate_dx_dy_arrow(x, y, direction, speed, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='black', width=0.25, shape='full'))
        
        
        # Football location
        footballX = playFootball.query('time == ' + str(time))['x']
        footballY = playFootball.query('time == ' + str(time))['y']
        patch.extend(plt.plot(footballX, footballY, 'o', c='black', ms=10, mec='white', zorder=3, data=playFootball.query('time == ' + str(time))['team']))
        # Field control of the different teams 
        x = np.linspace(0,120, 120)
        y = np.linspace(0,53.3,53)
        X,Y = np.meshgrid(x,y)
        
        mapping, home_, away_ = timeframe(play.query('frameId== ' + str(time)))
        
        patch.extend(plt.contourf(X, Y, mapping, cmap=cm.viridis_r, alpha=0.5).collections)
        
        return patch
    
    ims = [[]]
    for time in np.arange(minTime, maxTime+1):
        patch = update_animation(time)
        ims.append(patch)
        
    anim = animation.ArtistAnimation(fig, ims, repeat=False, blit=False)
    
    return anim

In [ ]:
anim = animate_player_movement(weekMod, 75, 2018090600)
writer = FFMpegWriter(fps=10)
anim.save('animation_notrail.mp4', writer=writer)
Video("animation_notrail.mp4")

In [ ]:
# Field control of the different teams 
x = np.linspace(0,120, 120)
y = np.linspace(0,53.3,53)
X,Y = np.meshgrid(x,y)
time = 45
mapping, home_, away_ = timeframe(play.query('frameId == ' + str(time)))

a = plt.contourf(X, Y, mapping, cmap=cm.viridis_r).collections

In [ ]:
def animate_field_control(weekdata, playId, gameId):
    play = weekMod.query('gameId== '+str(gameId)+' and playId=='+ str(playId))
    def update_animation(time):
        # Field control of the different teams 
        x = np.linspace(0,120, 120)
        y = np.linspace(0,53.3,53)
        X,Y = np.meshgrid(x,y)
        
        mapping, home_, away_ = timeframe(weekdata.query('frameId== ' + str(time)))
        
        patch=plt.contourf(X, Y, mapping, cmap=cm.viridis_r).collections
        return patch 
    
    ims = []
    for time in np.arange(minTime, maxTime+1):
        patch = update_animation(time)
        ims.append(patch)
        
    anim = animation.ArtistAnimation(fig, ims, repeat=False, blit=False)
    
    return anim

In [ ]:
anim = animate_player_movement(weekMod, 75, 2018090600)
writer = FFMpegWriter(fps=10)
anim.save('animation_field_control.mp4', writer=writer)
Video("animation_field_control.mp4")